In [1]:
from surprise import Dataset
from surprise import Dataset, Reader
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
import pickle

In [2]:
import pickle
import pandas as pd

In [3]:
import gzip
import tracemalloc


In [4]:
with open('modelo_entrenado.pkl', 'rb') as file:
    model = pickle.load(file)

In [5]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [6]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

In [7]:
entrenar = pd.read_csv(r'data_set_limpio\\modelo.csv.gz')

In [18]:
'Freejam' in entrenar['user'].unique()

False

In [45]:
prueba1 = pd.merge(opinion[['item_id','user','recommend','opinion','helpful']], df[['id','app_name']], left_on='item_id', right_on='id')

prueba1['recommend'] = prueba1['recommend'].replace({True: 1, False: 0})

prueba1.loc[prueba1['helpful'] == 'No ratings yet', 'helpful'] = 0

prueba1.loc[prueba1['helpful']  != 0 , 'helpful'] = 1

In [25]:
prueba1.loc[prueba1['user'] == 'inorisanbaka']

,item_id,user,recommend,opinion,helpful,id,app_name
5004,730,inorisanbaka,0,1,1,730,Counter-Strike: Global Offensive


In [393]:
for index, row in prueba1.iterrows():
    if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 1) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 0
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    else:
        prueba1.at[index, 'rating'] = 2
        

C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.

In [413]:
entrenar = prueba1[['user','app_name','rating']]

In [17]:
entrenar[20:40]

,item_id,user,recommend,opinion,id,app_name,rating
20,1250,76561198065764432,1,1,1250,Killing Floor,2.0
21,1250,usaidwotnow,1,2,1250,Killing Floor,3.0
22,1250,76561198081529182,1,2,1250,Killing Floor,3.0
23,1250,76561198047489572,1,2,1250,Killing Floor,3.0
24,1250,you_re_ded,1,0,1250,Killing Floor,1.0
25,1250,SwinburnCyphenMisslemike,1,1,1250,Killing Floor,2.0
26,1250,76561198080824028,1,1,1250,Killing Floor,2.0
27,1250,76561198053085687,1,0,1250,Killing Floor,1.0
28,1250,76561198097071063,1,0,1250,Killing Floor,1.0
29,1250,76561198064343153,0,0,1250,Killing Floor,0.0


In [438]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import train_test_split
import pandas as pd

# Supongamos que tienes un DataFrame 'df' con tus datos
# Columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Adaptar rating_scale según tu escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)  # Puedes ajustar el tamaño de prueba

# Inicializar el modelo SVD++
model = SVDpp(n_factors = 50, n_epochs = 25, lr_all= 0.005, reg_all= 0.04)  # Estos son valores de ejemplo

# Entrenar el modelo
model.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Evaluar las predicciones (por ejemplo, calcular el RMSE)
from surprise import accuracy
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 1.0196
RMSE: 1.0196406487549206


In [11]:
def recomendaciones_usuario(usuario):
        
    juegos_valorados = entrenar[entrenar['user'] == usuario]['app_name'].unique()

    todos_los_juegos = entrenar['app_name'].unique()

    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
    
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 

    diccionario = {}
    
    for recomendacion in recomendaciones:
        
        diccionario[recomendacion.iid] = recomendacion.est
        
    return diccionario

In [13]:
testeo1 = recomendaciones_usuario('Freejam')
testeo1

{'Batman - The Telltale Series': 3.3127410178425616,
 'Bit Blaster XL': 3.2488879421204144,
 'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™': 3.2435089824769547,
 'FINAL FANTASY TYPE-0™ HD': 3.1749564347814707,
 'Resident Evil 6 / Biohazard 6': 3.1636861429884084}

In [10]:
usuarios = entrenar['user'].unique()

In [19]:
diccionario = {}

for x in usuarios[:30]:
    
    diccionario[x] = recomendaciones_usuario(x)

diccionario

NameError: name 'recomendaciones_usuario' is not defined

In [15]:
entrenar.loc[entrenar['user'] == 'vinquility']

,item_id,user,recommend,opinion,id,app_name,rating
37704,413150,vinquility,1,1,413150,Stardew Valley,2.0
49594,431510,vinquility,1,2,431510,Mystic Destinies: Serendipity of Aeons,3.0


In [21]:
prueba1.to_csv('data_set_limpio/modelo.csv',encoding='UTF-8',index=False)

In [22]:
with open('data_set_limpio/modelo.csv', 'rb') as f_in, gzip.open('data_set_limpio/modelo.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)

In [425]:
with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(model, file)

Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.04}

In [361]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Supongamos que 'entrenar' es tu DataFrame con las columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Ajustar según la escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Definir el espacio de parámetros a explorar
'''param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'init_mean': [0, 0.5],
    'init_std_dev': [0, 0.5],
    'random_state': [0, 42],  # Cambia los valores de la semilla si es necesario
    'verbose': [True, False]
}'''
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 25],  # Reducir el rango de epochs
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'random_state': [0, 42]
}

# Inicializar el modelo SVD++
model = SVDpp()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

# Ajustar la búsqueda de cuadrícula con los datos
grid_search.fit(data)

# Obtener los mejores resultados
best_params = grid_search.best_params['rmse']
print("Mejores parámetros encontrados:", best_params)

#Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.04, 'random_state': 42}

Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.04, 'random_state': 0}


In [362]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Supongamos que 'entrenar' es tu DataFrame con las columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Ajustar según la escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Definir el espacio de parámetros a explorar
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'init_mean': [0, 0.5],
    'init_std_dev': [0, 0.5],
    'random_state': [0, 42],  # Cambia los valores de la semilla si es necesario
    'verbose': [True, False]
}
'''param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 25],  # Reducir el rango de epochs
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'random_state': [0, 42]
}'''

# Inicializar el modelo SVD++
model = SVDpp()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

# Ajustar la búsqueda de cuadrícula con los datos
grid_search.fit(data)

# Obtener los mejores resultados
best_params = grid_search.best_params['rmse']
print("Mejores parámetros encontrados:", best_params)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8


In [20]:

def recomendaciones_usuario(usuario : str):
    
    if usuario not in entrenar['user'].unique():
        return {'El usuario no existe en la base de datos'}
    else:
        diccionario = {}
        
        juegos_valorados = entrenar[entrenar['user'] == usuario]['app_name'].unique()

        todos_los_juegos = entrenar['app_name'].unique()

        juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

        predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
        
        recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 
        cont = 1
        for recomendacion in recomendaciones:
            diccionario[f'Top {cont}'] = recomendacion.iid
            cont+=1
        return diccionario

In [9]:
testeo1 = recomendaciones_usuario('Riot-Punch')
testeo1

{'Usuario: Riot-Punch': {'El usuario Riot-Punch no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.': ['Portal 2',
   'Skullgirls',
   'Arma 2: Operation Arrowhead',
   'The Expendabros',
   'Half-Life 2']}}

In [8]:
import random

def recomendaciones_usuario(usuario: str):
    if usuario not in entrenar['user'].unique():
        juegos_aleatorios = list(entrenar['app_name'].sample(5))
        mensaje = f'El usuario {usuario} no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.'
        return {mensaje: juegos_aleatorios}
    else:
        diccionario = {}
        
        juegos_valorados = entrenar[entrenar['user'] == usuario]['app_name'].unique()
        todos_los_juegos = entrenar['app_name'].unique()

        juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

        predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
        
        recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 
        cont = 1
        for recomendacion in recomendaciones:
            diccionario[f'Top {cont}'] = recomendacion.iid
            cont += 1
        return {f'Usuario: {usuario}': diccionario}

